In [2]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re

conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_collection = actor_db.actor_collection

actors_info_list = list()

cine21_url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section'] = 'actor'
post_data['period_start'] = '2018-08'
post_data['gender'] = 'all'

for index in range(1, 21):
    post_data['page'] = index

    res = requests.post(cine21_url, data=post_data)
    soup = BeautifulSoup(res.content, 'html.parser')

    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info > li > strong')
    movies = soup.select('ul.mov_list')
    rankings = soup.select('li.people_li > span.grade')
    
    for index, actor in enumerate(actors):
        actor_name = re.sub('\(\w*\)', '', actor.text)
        actor_hits = int(hits[index].text.replace(',', ''))
        movie_titles = movies[index].select('li a span')
        movie_title_list = list()
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        actor_info_dict = dict()
        actor_info_dict['배우이름'] = actor_name
        actor_info_dict['흥행지수'] = actor_hits
        actor_info_dict['출연영화'] = movie_title_list
        actor_info_dict['랭킹'] = rankings[index].text

        actor_link = 'http://www.cine21.com' + actor.select_one('a').attrs['href']
        response_actor = requests.get(actor_link)
        soup_actor = BeautifulSoup(response_actor.content, 'html.parser')
        default_info = soup_actor.select_one('ul.default_info')
        actor_details = default_info.select('li')

        for actor_item in actor_details:
            actor_item_field = actor_item.select_one('span.tit').text
            actor_item_value = re.sub('<span.*?>.*?</span>', '', str(actor_item))
            actor_item_value = re.sub('<.*?>', '', actor_item_value)
            actor_info_dict[actor_item_field] = actor_item_value
        actors_info_list.append(actor_info_dict)
        
actor_collection.insert_many(actors_info_list) 

In [ ]:
actor_info_list

In [ ]:
actor_collection.insert_many(actor_info_list)

In [4]:
results = actor_collection.find()
for r in results:
    print(r)

{'_id': ObjectId('62036a59ebbd36b5409bf7d4'), '배우이름': '하정우', '흥행지수': 21830, '출연영화': ['신과 함께-인과 연', '백두산', 'PMC: 더 벙커', '클로젯', '신과 함께-죄와 벌', '멋진 하루'], '랭킹': '1', '다른 이름': '김성훈; 河正宇', '직업': '배우', '생년월일': '1978-03-11', '성별': '남', '홈페이지': '\nhttps://www.facebook.com/ft.hajungwoo\n', '신장/체중': '184cm, 75kg', '학교': '중앙대학교 연극학 학사', '취미': '피아노, 검도, 수영'}
{'_id': ObjectId('62036a59ebbd36b5409bf7d5'), '배우이름': '마동석', '흥행지수': 19552, '출연영화': ['나쁜 녀석들: 더 무비', '신과 함께-인과 연', '성난황소', '동네사람들', '원더풀 고스트', '노리개: 그녀의 눈물'], '랭킹': '2', '다른 이름': 'Ma Dongseok', '직업': '배우', '생년월일': '1971-03-01', '성별': '남', '홈페이지': '\nhttps://www.instagram.com/madongseok_/\nhttps://twitter.com/madongseok12\n', '신장/체중': '178cm, 100kg'}
{'_id': ObjectId('62036a59ebbd36b5409bf7d6'), '배우이름': '이병헌', '흥행지수': 16450, '출연영화': ['백두산', '남산의 부장들', '내부자들', '그것만이 내 세상', '광해, 왕이 된 남자', '번지점프를 하다'], '랭킹': '3', '다른 이름': 'Byung-hun Lee;BH Lee', '직업': '배우', '생년월일': '1970-07-12', '성별': '남', '홈페이지': '\nhttp://www.leebyunghun.kr/\n', '신장/체중': '177cm, 7

In [5]:
import pandas as pd
pd.DataFrame(actors_info_list)

,배우이름,흥행지수,출연영화,랭킹,다른 이름,직업,생년월일,성별,홈페이지,신장/체중,학교,취미,_id,특기,소속사,원어명
0,하정우,21830,"[신과 함께-인과 연, 백두산, PMC: 더 벙커, 클로젯, 신과 함께-죄와 벌, ...",1,김성훈; 河正宇,배우,1978-03-11,남,\nhttps://www.facebook.com/ft.hajungwoo\n,"184cm, 75kg",중앙대학교 연극학 학사,"피아노, 검도, 수영",62036a59ebbd36b5409bf7d4,NaN,NaN,NaN
1,마동석,19552,"[나쁜 녀석들: 더 무비, 신과 함께-인과 연, 성난황소, 동네사람들, 원더풀 고스...",2,Ma Dongseok,배우,1971-03-01,남,\nhttps://www.instagram.com/madongseok_/\nhttp...,"178cm, 100kg",NaN,NaN,62036a59ebbd36b5409bf7d5,NaN,NaN,NaN
2,이병헌,16450,"[백두산, 남산의 부장들, 내부자들, 그것만이 내 세상, 광해, 왕이 된 남자, 번...",3,Byung-hun Lee;BH Lee,배우,1970-07-12,남,\nhttp://www.leebyunghun.kr/\n,"177cm, 72kg",한양대학교 불어문학과,"모자수집, 여행",62036a59ebbd36b5409bf7d6,"태권도, 스노우보드, 수영, 팔씨름",NaN,NaN
3,황정민,15902,"[공작, 인질, 다만 악에서 구하소서, 신세계, 와이키키 브라더스, 부당거래]",4,NaN,배우,1970-09-01,남,NaN,"180cm, 75kg",서울예술대학 연극과 졸업,NaN,62036a59ebbd36b5409bf7d7,"농구, 악기연주",예당엔터테인먼트,NaN
4,이성민,15648,"[남산의 부장들, 목격자, 공작, 기적, 비스트, 마약왕]",5,NaN,배우,1968-10-15,남,NaN,178cm,NaN,NaN,62036a59ebbd36b5409bf7d8,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,김혜수,1953,"[내가 죽던 날, 국가부도의 날, 타짜, 도둑들, 닥터봉]",136,NaN,배우,1970-09-05,여,NaN,"171cm, 49kg",동국대 연극영화 - 성균관대언론정보대학원 석사,"영화 감상, 사진집 모으기, 태권도, 수영, 테니스, 볼링",62036a59ebbd36b5409bf85b,태권도,NaN,NaN
136,남문철,1950,"[공작, 나랏말싸미, 유열의 음악앨범, 독전, 애비규환, 4등]",137,NaN,배우,1971-03-20,남,NaN,NaN,NaN,NaN,62036a59ebbd36b5409bf85c,NaN,NaN,NaN
137,손종학,1946,"[돈, 강철비2: 정상회담, 정직한 후보, 도희야, 검은 사제들, 비밀은 없다]",138,NaN,배우,1967-06-20,남,NaN,NaN,NaN,NaN,62036a59ebbd36b5409bf85d,NaN,NaN,NaN
138,최덕문,1911,"[나랏말싸미, 마약왕, 블랙머니, 소공녀, 암살, 애비규환]",139,NaN,배우,1970-00-00,남,NaN,NaN,NaN,NaN,62036a59ebbd36b5409bf85e,NaN,NaN,NaN
